# LangChain 核心模块学习：Chains

对于简单的大模型应用，单独使用语言模型（LLMs）是可以的。

**但更复杂的大模型应用需要将 `LLMs` 和 `Chat Models` 链接在一起 - 要么彼此链接，要么与其他组件链接。**

LangChain 为这种“链式”应用程序提供了 `Chain` 接口。

LangChain 以通用方式定义了 `Chain`，它是对组件进行调用序列的集合，其中可以包含其他链。

In [2]:
! pip install -U langchain

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     ---------------------------------------- 0.0/990.0 kB ? eta -:--:--
     ---------------------------------------- 10.2/990.0 kB ? eta -:--:--
     ------ ------------------------------- 174.1/990.0 kB 2.6 MB/s eta 0:00:01
     ------ ------------------------------- 174.1/990.0 kB 2.6 MB/s eta 0:00:01
     ---------------- --------------------- 440.3/990.0 kB 2.8 MB/s eta 0:00:01
     ---------------- --------------------- 440.3/990.0 kB 2.8 MB/s eta 0:00:01
     ---------------- --------------------- 440.3/990.0 kB 2.8 MB/s eta 0:00:01
     ---------------- --------------------- 440.3/990.0 kB 2.8 MB/s eta 0:00:01
     -------------------- ----------------- 532.5/990.0 kB 1.6 MB/s eta 0:00:01
     --------------------------------- ---- 880.6/990.0 kB 2.5 MB/s eta 0:00:01
     ----------------------------------- -- 931.8/990.0 kB 2.5 MB/s eta 0:00:01
     -------------------------------------  972.8/990.0 kB 2.2 MB/s eta 

## Chain Class 基类

类继承关系：

```
Chain --> <name>Chain  # Examples: LLMChain, MapReduceChain, RouterChain
```

**代码实现：https://github.com/langchain-ai/langchain/blob/master/libs/langchain/langchain/chains/base.py**

```python
# 定义一个名为Chain的基础类
class Chain(Serializable, Runnable[Dict[str, Any], Dict[str, Any]], ABC):
    """为创建结构化的组件调用序列的抽象基类。
    
    链应该用来编码对组件的一系列调用，如模型、文档检索器、其他链等，并为此序列提供一个简单的接口。
    
    Chain接口使创建应用程序变得容易，这些应用程序是：
    - 有状态的：给任何Chain添加Memory可以使它具有状态，
    - 可观察的：向Chain传递Callbacks来执行额外的功能，如记录，这在主要的组件调用序列之外，
    - 可组合的：Chain API足够灵活，可以轻松地将Chains与其他组件结合起来，包括其他Chains。
    
    链公开的主要方法是：
    - `__call__`：链是可以调用的。`__call__`方法是执行Chain的主要方式。它将输入作为一个字典接收，并返回一个字典输出。
    - `run`：一个方便的方法，它以args/kwargs的形式接收输入，并将输出作为字符串或对象返回。这种方法只能用于一部分链，不能像`__call__`那样返回丰富的输出。
    """

    # 调用链
    def invoke(
        self, input: Dict[str, Any], config: Optional[runnableConfig] = None
    ) -> Dict[str, Any]:
        """传统调用方法。"""
        return self(input, **(config or {}))

    # 链的记忆，保存状态和变量
    memory: Optional[BaseMemory] = None
    """可选的内存对象，默认为None。
    内存是一个在每个链的开始和结束时被调用的类。在开始时，内存加载变量并在链中传递它们。在结束时，它保存任何返回的变量。
    有许多不同类型的内存，请查看内存文档以获取完整的目录。"""

    # 回调，可能用于链的某些操作或事件。
    callbacks: Callbacks = Field(default=None, exclude=True)
    """可选的回调处理程序列表（或回调管理器）。默认为None。
    在对链的调用的生命周期中，从on_chain_start开始，到on_chain_end或on_chain_error结束，都会调用回调处理程序。
    每个自定义链可以选择调用额外的回调方法，详细信息请参见Callback文档。"""

    # 是否详细输出模式
    verbose: bool = Field(default_factory=_get_verbosity)
    """是否以详细模式运行。在详细模式下，一些中间日志将打印到控制台。默认值为`langchain.verbose`。"""

    # 与链关联的标签
    tags: Optional[List[str]] = None
    """与链关联的可选标签列表，默认为None。
    这些标签将与对这个链的每次调用关联起来，并作为参数传递给在`callbacks`中定义的处理程序。
    你可以使用这些来例如识别链的特定实例与其用例。"""

    # 与链关联的元数据
    metadata: Optional[Dict[str, Any]] = None
    """与链关联的可选元数据，默认为None。
    这些元数据将与对这个链的每次调用关联起来，并作为参数传递给在`callbacks`中定义的处理程序。
    你可以使用这些来例如识别链的特定实例与其用例。"""
```

## LLMChain

LLMChain 是 LangChain 中最简单的链，作为其他复杂 Chains 和 Agents 的内部调用，被广泛应用。

一个LLMChain由PromptTemplate和语言模型（LLM or Chat Model）组成。它使用直接传入（或 memory 提供）的 key-value 来规范化生成 Prompt Template（提示模板），并将生成的 prompt （格式化后的字符串）传递给大模型，并返回大模型输出。

![](../images/llm_chain.png)

In [9]:
import os
os.environ['OPENAI_API_KEY'] = 'XXX'
os.environ['OPENAI_BASE_URL'] = 'XXX'

In [10]:
from langchain_openai import OpenAI
from langchain.prompts import PromptTemplate

llm = OpenAI(model_name="gpt-3.5-turbo-instruct", temperature=0.9, max_tokens=500)

In [11]:
prompt = PromptTemplate(
    input_variables=["product"],
    template="给制造{product}的有限公司取10个好名字，并给出完整的公司名称",
)

In [12]:
from langchain.chains import LLMChain

chain = LLMChain(llm=llm, prompt=prompt)
print(chain.invoke({
    'product': "性能卓越的GPU"
    }))

{'product': '性能卓越的GPU', 'text': '.\n\n`01.` 莱弗科特科技有限公司\n\n`02.` 雷神之眼科技有限公司\n\n`03.` 无极创新科技有限公司\n\n`04.` 奇点视界科技有限公司\n\n`05.` 璀璨阳光科技有限公司\n\n`06.` 飞驰科技有限公司\n\n`07.` 千兆科技有限公司\n\n`08.` 银河探索科技有限公司\n\n`09.` 巨人之力科技有限公司\n\n`10.` 神威创新科技有限公司'}


In [13]:
chain.verbose =True

In [14]:
chain.verbose

True

In [15]:
print(chain.invoke({
    'product': "性能卓越的GPU"
    }))



> Entering new LLMChain chain...
Prompt after formatting:
给制造性能卓越的GPU的有限公司取10个好名字，并给出完整的公司名称

> Finished chain.
{'product': '性能卓越的GPU', 'text': '\n\n1.Impulse Graphics Company\nImpulse图形公司\n\n2.ElevateFX Corporation\nElevateFX有限公司\n\n3.VortexTech Technologies\nVortexTech技术公司\n\n4.PrecisionPixel Inc.\nPrecisionPixel股份有限公司\n\n5.InnovateGPU Solutions\nInnovateGPU解决方案\n\n6.NeoDynasty Dynamics\nNeoDynasty动力学\n\n7.Pantheon Processing Co.\nPantheon处理有限公司\n\n8.AstroByte Innovations\nAstroByte创新公司\n\n9.GenesisGraphics Enterprise\nGenesisGraphics企业\n\n10.AuroraTech Industries\nAuroraTech工业公司'}


## Sequential Chain

串联式调用语言模型（将一个调用的输出作为另一个调用的输入）。

顺序链（Sequential Chain ）允许用户连接多个链并将它们组合成执行特定场景的流水线（Pipeline）。有两种类型的顺序链：

- SimpleSequentialChain：最简单形式的顺序链，每个步骤都具有单一输入/输出，并且一个步骤的输出是下一个步骤的输入。
- SequentialChain：更通用形式的顺序链，允许多个输入/输出。

### 使用 SimpleSequentialChain 实现戏剧摘要和评论（单输入/单输出）

![](../images/simple_sequential_chain_0.png)

In [20]:
# 这是一个 LLMChain，用于根据剧目的标题撰写简介。

llm = OpenAI(temperature=0.7, max_tokens=1000)

template = """你是一位剧作家。根据戏剧的标题，你的任务是为该标题写一个简介。

标题：{title}
剧作家：以下是对上述戏剧的简介："""

prompt_template = PromptTemplate(input_variables=["title"], template=template)
synopsis_chain = LLMChain(llm=llm, prompt=prompt_template)

In [21]:
# 这是一个LLMChain，用于根据剧情简介撰写一篇戏剧评论。
# llm = OpenAI(temperature=0.7, max_tokens=1000)
template = """你是《纽约时报》的戏剧评论家。根据剧情简介，你的工作是为该剧撰写一篇评论。

剧情简介：
{synopsis}

以下是来自《纽约时报》戏剧评论家对上述剧目的评论："""

prompt_template = PromptTemplate(input_variables=["synopsis"], template=template)
review_chain = LLMChain(llm=llm, prompt=prompt_template)

![](../images/simple_sequential_chain_1.png)

In [22]:
# 这是一个SimpleSequentialChain，按顺序运行这两个链
from langchain.chains import SimpleSequentialChain

overall_chain = SimpleSequentialChain(chains=[synopsis_chain, review_chain], verbose=True)

In [23]:
review = overall_chain.invoke("三体人不是无法战胜的")



> Entering new SimpleSequentialChain chain...


《三体人不是无法战胜的》是一部关于人类与外星种族的生死对决的戏剧。故事发生在一个虚构的未来世界，人类正在面临一场来自三体星系的入侵。这个外星种族拥有强大的科技和战斗力，一度让人类感到绝望。

然而，主角们并不认为三体人是无法战胜的。他们相信通过勇气、智慧和团结，人类有能力面对挑战并最终取得胜利。在经历了一系列惊险的战斗和牺牲之后，人类终于找到了击败三体人的方法。

除了外星入侵的故事线，该剧还深入探讨了人类的内心世界和情感。主角们的友谊、爱情和牺牲都为戏剧增添了更多的温情和感动。

《三体人不是无法战胜的》将带领观众进入一个充满惊险、感动和希望的战斗世界，同时也会让观众思考人类的力量和价值。这部戏剧将带来一场视听盛宴，让观众体验超越想象的冒险之旅。


《三体人不是无法战胜的》是一部令人难忘的戏剧作品，它将观众带入了一个充满惊险和感动的外星入侵世界。但这部戏剧不仅仅是一场视听盛宴，它更深层次地探讨了人类的力量和价值。

故事情节紧张刺激，让观众不能释怀地投入其中。主角们的勇气、智慧和团结让人们相信，即使面对强大的敌人，人类也能够战胜困难。同时，剧中对人类内心世界和情感的描绘也让观众感受到更多的温情和感动。

该剧的制作精良，舞台设计和特效令人惊叹。每一场战斗的场景都让人身临其境，让观众感受到超越想象的冒险之旅。演员们的表演也非常出色，他们生动地诠释了每个角色的情感和内心挣扎。

总的来说，《三体人不是无法战胜的》是一部充满惊喜和感动的戏剧作品。它不仅仅是一场娱乐，更是一次关于人类力量和价值的思考。我强烈推荐观众们去体验这部作品，它将带给你们一场难忘的冒险之旅。

> Finished chain.


In [24]:
review = overall_chain.invoke("星球大战第九季")



> Entering new SimpleSequentialChain chain...


《星球大战第九季》是一部充满惊险刺激的科幻戏剧。它延续了前八部《星球大战》的故事，将带领观众进入一个充满未知的宇宙世界。在这个世界里，人类与外星种族之间的战争仍在继续，但随着时间的推移，敌对双方的角色和立场也发生了巨大的变化。

在这部戏剧中，观众将见证英雄们的勇气和决心，以及他们面对的艰难抉择。同时，也将揭开隐藏在背后的阴谋和秘密。随着故事的发展，观众将被带入一场精彩的星际大战，亲眼目睹双方之间的激烈战斗和惊心动魄的决战。

除了刺激的战斗场面，本剧也不乏感人的情感戏。在这个充满战争和冲突的宇宙中，爱情和友情的力量也将被赋予重要的意义。

《星球大战第九季》将为观众带来一场视觉和心灵的双重冲击。它将探讨人类的本性、权力的诱惑以及永恒的正义与邪恶之间的斗争。故事的结局将给观众留下深刻的思考，并为整个系列画上完美的句号。敬请期待这场惊险刺激的星际大战，只在《星球大战第九季》！


《星球大战第九季》是一部令人兴奋的科幻戏剧，它延续了《星球大战》系列的传统，为观众呈现了一场视觉和心灵的双重冲击。导演和演员们精湛的表演，加上精彩的剧情，让观众仿佛置身于一个充满未知和惊险的宇宙世界。

这部戏剧除了充满刺激的战斗场面外，也有着感人的情感戏。在这个充满战争和冲突的宇宙中，爱情和友情成为了人们唯一的寄托，也让故事更加丰富和动人。

《星球大战第九季》也探讨了许多深刻的主题，包括人类的本性、权力的诱惑以及正义与邪恶的斗争。观众在欣赏精彩的特效和动作场面的同时，也会被故事所打动，思考这些重要的问题。

总的来说，《星球大战第九季》是一部令人难以忘怀的科幻戏剧，它给观众带来了一场精彩的星际大战，也为整个系列画上了完美的句号。强烈推荐所有的《星球大战》粉丝和科幻爱好者观看这部作品，你们一定会被它所吸引和感动。

> Finished chain.


### 使用 SequentialChain 实现戏剧摘要和评论（多输入/多输出）

![](../images/sequential_chain_0.png)

In [25]:
# # 这是一个 LLMChain，根据剧名和设定的时代来撰写剧情简介。
llm = OpenAI(temperature=.7, max_tokens=1000)
template = """你是一位剧作家。根据戏剧的标题和设定的时代，你的任务是为该标题写一个简介。

标题：{title}
时代：{era}
剧作家：以下是对上述戏剧的简介："""

prompt_template = PromptTemplate(input_variables=["title", "era"], template=template)
# output_key
synopsis_chain = LLMChain(llm=llm, prompt=prompt_template, output_key="synopsis", verbose=True)

In [26]:
# 这是一个LLMChain，用于根据剧情简介撰写一篇戏剧评论。

template = """你是《纽约时报》的戏剧评论家。根据该剧的剧情简介，你需要撰写一篇关于该剧的评论。

剧情简介：
{synopsis}

来自《纽约时报》戏剧评论家对上述剧目的评价："""

prompt_template = PromptTemplate(input_variables=["synopsis"], template=template)
review_chain = LLMChain(llm=llm, prompt=prompt_template, output_key="review", verbose=True)

In [27]:
from langchain.chains import SequentialChain

m_overall_chain = SequentialChain(
    chains=[synopsis_chain, review_chain],
    input_variables=["era", "title"],
    # Here we return multiple variables
    output_variables=["synopsis", "review"],
    verbose=True)

In [28]:
m_overall_chain.invoke({"title":"三体人不是无法战胜的", "era": "二十一世纪的新中国"})



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
你是一位剧作家。根据戏剧的标题和设定的时代，你的任务是为该标题写一个简介。

标题：三体人不是无法战胜的
时代：二十一世纪的新中国
剧作家：以下是对上述戏剧的简介：

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
你是《纽约时报》的戏剧评论家。根据该剧的剧情简介，你需要撰写一篇关于该剧的评论。

剧情简介：


《三体人不是无法战胜的》讲述的是二十一世纪的新中国，人类社会与外星种族“三体人”之间的一场生死搏斗。在这个时代，人类已经实现了科技和社会的巨大进步，但是也因此吸引了来自宇宙的三体人的注意。

三体人拥有超越人类想象的科技能力，他们看不起人类，认为人类只是一个微不足道的存在。为了保护自己的家园，人类必须要与三体人展开一场殊死搏斗。在这场战争中，人类学会了团结和勇气，他们用自己的智慧和勇气，挑战了三体人的强大。

在战争的过程中，人类也发现了三体人的弱点，他们并不是无法战胜的。最终，人类战胜了三体人，守护了自己的家园。

这部戏剧通过对人类面对强大外敌时的勇气和智慧的赞颂，向观众展现了人类的坚韧和不屈不挠的精神。同时，也提醒我们，在面对挑战时，团结和勇气是战胜一切困难的关键。

来自《纽约时报》戏剧评论家对上述剧目的评价：

> Finished chain.

> Finished chain.


{'title': '三体人不是无法战胜的',
 'era': '二十一世纪的新中国',
 'synopsis': '\n\n《三体人不是无法战胜的》讲述的是二十一世纪的新中国，人类社会与外星种族“三体人”之间的一场生死搏斗。在这个时代，人类已经实现了科技和社会的巨大进步，但是也因此吸引了来自宇宙的三体人的注意。\n\n三体人拥有超越人类想象的科技能力，他们看不起人类，认为人类只是一个微不足道的存在。为了保护自己的家园，人类必须要与三体人展开一场殊死搏斗。在这场战争中，人类学会了团结和勇气，他们用自己的智慧和勇气，挑战了三体人的强大。\n\n在战争的过程中，人类也发现了三体人的弱点，他们并不是无法战胜的。最终，人类战胜了三体人，守护了自己的家园。\n\n这部戏剧通过对人类面对强大外敌时的勇气和智慧的赞颂，向观众展现了人类的坚韧和不屈不挠的精神。同时，也提醒我们，在面对挑战时，团结和勇气是战胜一切困难的关键。',
 'review': '\n\n《三体人不是无法战胜的》是一部令人振奋的戏剧作品。它不仅仅是一场关于科技和外星种族的对抗，更是对人类精神的歌颂。\n\n该剧将观众带入一个虚拟的未来世界，展现了人类在面对强大外敌时的勇气和智慧。通过对战争的描写，观众不仅可以感受到人类的坚韧和不屈不挠，也可以看到他们在团结和勇气面前的强大力量。\n\n同时，该剧也提醒我们，面对挑战时，团结和勇气是战胜一切困难的关键。在这部作品中，人类团结一心，用自己的智慧和勇气战胜了超越想象的外星种族，守护了自己的家园。这无疑是对人类坚韧精神的最大褒奖。\n\n此外，该剧也通过对三体人的描写，展现了人类的自信和自主性。尽管面对强大的外星种族，人类仍然保持着自己的独特性和价值观，并最终发现了对方的弱点，取得了胜利。这也提醒我们，自信和自主性是人类文明不断进步的重要因素。\n\n总的来说，《三体人不是无法战胜的》是一部充满希望和勇气的戏剧作品。它向我们展示了人类面对挑战时的勇气和智慧，也提醒我们团结和自信的重要性。相信这部作品会给观众带来深刻的思考和启发。'}

### Homework

#### 使用 OutputParser 优化 overall_chain 输出格式，区分 synopsis_chain 和 review_chain 的结果

### Homework

#### 扩展langchain chains
##### 扩展chains使其支持生物计算机和汉语文学老师等学科的提示词模版及对应CHAINS问答。

In [ ]:
## Chain

In [36]:
# # 这是一个 LLMChain，根据学科名生成学科老师。
llm = OpenAI(temperature=.7, max_tokens=1000)
template = """你是一位学科老师。根据传入的学科及学科问题，对问题做出回答。

学科：{subject}
问题：{question}
答案：以下是对上述问题的回答："""

prompt_template = PromptTemplate(input_variables=["subject", "question"], template=template)
# output_key
synopsis_chain = LLMChain(llm=llm, prompt=prompt_template, output_key="synopsis", verbose=True)

In [39]:
print(synopsis_chain.invoke({
    'subject': "生物",
    'question': "什么是光合作用的化学方程式？描述光合作用过程中光反应和暗反应（卡尔文循环）的主要步骤。"
    }))



> Entering new LLMChain chain...
Prompt after formatting:
你是一位学科老师。根据传入的学科及学科问题，对问题做出回答。

学科：生物
问题：什么是光合作用的化学方程式？描述光合作用过程中光反应和暗反应（卡尔文循环）的主要步骤。
答案：以下是对上述问题的回答：

> Finished chain.
{'subject': '生物', 'question': '什么是光合作用的化学方程式？描述光合作用过程中光反应和暗反应（卡尔文循环）的主要步骤。', 'synopsis': '\n\n光合作用是指植物和某些细菌利用光能将二氧化碳和水转化为有机物质的过程。其化学方程式可以表示为：6CO2 + 6H2O + 光能 → C6H12O6 + 6O2。\n\n光合作用包括光反应和暗反应两个阶段。光反应发生在叶绿体的光合作用膜中，其主要步骤如下：\n1. 光能被叶绿素吸收，激发电子从光合作用复合物I和II中释放出来。\n2. 电子经过一系列传递过程，最终被光合作用复合物I和II捕获，生成ATP和NADPH。\n3. 水分子被光合作用复合物II分解，释放出氧气。\n\n暗反应也被称为卡尔文循环，发生在叶绿体基质中，主要步骤如下：\n1. CO2通过叶片上的气孔进入叶绿体，与ATP和NADPH反应，生成三碳糖（甘油醛-3-磷酸）。\n2. 三碳糖在一系列反应下转化为六碳糖（葡萄糖），同时再生ATP和NADPH。\n3. 六碳糖经过一系列转化反应，最终形成植物生长所需的有机物质。\n总的来说，光合作用是一个复杂的过程，需要光能、水和二氧化碳的共同作用，才能将无机物转化为有机物。同时，光反应和暗反应的相互配合也是实现光合作用的关键。'}


In [40]:
print(synopsis_chain.invoke({
    'subject': "计算机",
    'question': "什么是递归算法？请解释递归算法的基本概念，并提供一个经典的递归算法示例，如计算斐波那契数列的算法。详细描述该算法的实现步骤。"
    }))



> Entering new LLMChain chain...
Prompt after formatting:
你是一位学科老师。根据传入的学科及学科问题，对问题做出回答。

学科：计算机
问题：什么是递归算法？请解释递归算法的基本概念，并提供一个经典的递归算法示例，如计算斐波那契数列的算法。详细描述该算法的实现步骤。
答案：以下是对上述问题的回答：

> Finished chain.
{'subject': '计算机', 'question': '什么是递归算法？请解释递归算法的基本概念，并提供一个经典的递归算法示例，如计算斐波那契数列的算法。详细描述该算法的实现步骤。', 'synopsis': '\n\n递归算法是一种解决问题的方法，它在解决问题的过程中调用自身来解决更小规模的相同问题，直到问题规模缩小到可以直接解决的程度。简单来说，递归算法就是指一个函数或过程直接或间接地调用自身的算法。它的基本思想是将一个大问题分解为若干个相同的小问题，然后通过解决这些小问题来解决整个问题。\n\n一个经典的递归算法示例是计算斐波那契数列的算法。斐波那契数列是一个无限数列，第一个数和第二个数都是1，从第三项开始，每一项都等于前两项的和，即1、1、2、3、5、8、13、21、34……下面是该算法的实现步骤：\n\nStep1：确定递归函数的参数和返回值，对于斐波那契数列来说，参数为要计算的数列的项数n，返回值为第n项的值。\nStep2：编写递归终止条件，当n=1或n=2时，返回1。\nStep3：编写递归函数体，即当n>2时，返回递归调用自身两次的结果相加的值。\nStep4：调用递归函数，传入要计算的数列的项数n，获取结果。\nStep5：打印结果。\n\n例如要计算斐波那契数列的第10项的值，根据上述步骤，首先确定参数为n=10，返回值为第10项的值；其次，编写递归终止条件，当n=1或n=2时，返回1；然后编写递归函数体，即当n>2时，返回递归调用自身两次的结果相加的值，即f(n-1)+f(n-2)；最后调用递归函数，传入n=10，获取结果，即f(10)=f(9)+f(8)=f(8)+f(7)+f(7)+f(6)=……=f(2)+f(1)+f(1)+f(0)=1+1+1+1=8；最后打印结果为8，即斐波那契数列的第10项的值为8。\n\n总的来说，递归算法能够

In [41]:
print(synopsis_chain.invoke({
    'subject': "汉语文学",
    'question': "请解释《红楼梦》中“木石前盟”的典故及其在整部小说中的意义。"
    }))



> Entering new LLMChain chain...
Prompt after formatting:
你是一位学科老师。根据传入的学科及学科问题，对问题做出回答。

学科：汉语文学
问题：请解释《红楼梦》中“木石前盟”的典故及其在整部小说中的意义。
答案：以下是对上述问题的回答：

> Finished chain.
{'subject': '汉语文学', 'question': '请解释《红楼梦》中“木石前盟”的典故及其在整部小说中的意义。', 'synopsis': '《红楼梦》中的“木石前盟”典故来源于《史记·司马相如列传》中的故事，故事中梁孝王与楚王相遇，两人用木石为盟，表示两国友好。在《红楼梦》中，林黛玉和薛宝钗也用木石为盟，表达姐妹情谊。在整部小说中，这一典故体现了林黛玉和薛宝钗的深厚友谊，也预示着后来林黛玉和宝玉的爱情，但也暗示着两人最终的悲剧命运，因为木石易变，表明姐妹情谊和爱情都可能难以长久。同时，这一典故也反映出作者对封建礼教和命运的深刻思考。'}


In [ ]:
## SequentialChain

In [42]:
# # 这是一个 LLMChain，根据学科名生成学科问题。
llm = OpenAI(temperature=.7, max_tokens=1000)
template = """你是一位学生。根据传入的学科，提出一个对应学科的知识问题。

学科：{subject}
问题：以下是关于上述学科的一个知识问题："""

prompt_template = PromptTemplate(input_variables=["subject"], template=template)
# output_key
synopsis_chain = LLMChain(llm=llm, prompt=prompt_template, output_key="synopsis", verbose=True)

In [43]:
# 这是一个LLMChain，用于根据学科问题做出对应问题的答案。

template = """你是一位学科老师。根据传入的学科问题，你需要做出该学科问题的答案。

学科问题：
{synopsis}

来自学科老师对上述问题的答案："""

prompt_template = PromptTemplate(input_variables=["synopsis"], template=template)
review_chain = LLMChain(llm=llm, prompt=prompt_template, output_key="review", verbose=True)

In [44]:
from langchain.chains import SequentialChain

m_overall_chain = SequentialChain(
    chains=[synopsis_chain, review_chain],
    input_variables=["subject"],
    # Here we return multiple variables
    output_variables=["synopsis", "review"],
    verbose=True)

In [45]:
m_overall_chain.invoke({"subject":"生物"})



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
你是一位学生。根据传入的学科，提出一个对应学科的知识问题。

学科：生物
问题：以下是关于上述学科的一个只是问题：

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
你是一位学科老师。根据传入的学科问题，你需要做出该学科问题的答案。

学科问题：

生物学中的细胞是什么？

来自学科老师对上述问题的答案：

> Finished chain.

> Finished chain.


{'subject': '生物',
 'synopsis': '\n生物学中的细胞是什么？',
 'review': '细胞是生物学中的基本单位，是构成所有生物体的最小结构和功能单位。它具有自我复制、自我维持和遗传变异的能力，是生物体的基础组成部分。细胞可以分为原核细胞和真核细胞，具有不同的结构和功能。通过细胞间的相互作用，细胞可以完成生命的各种活动，维持生物体的正常运转。 '}

In [46]:
m_overall_chain.invoke({"subject":"计算机"})



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
你是一位学生。根据传入的学科，提出一个对应学科的知识问题。

学科：计算机
问题：以下是关于上述学科的一个只是问题：

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
你是一位学科老师。根据传入的学科问题，你需要做出该学科问题的答案。

学科问题：
请问计算机如何实现人工智能？



来自学科老师对上述问题的答案：

> Finished chain.

> Finished chain.


{'subject': '计算机',
 'synopsis': '请问计算机如何实现人工智能？\n\n',
 'review': '\n\n计算机实现人工智能主要依赖于两个关键技术：机器学习和自然语言处理。机器学习是指利用大量数据来训练计算机模型，使其能够自动学习并改进其性能，从而实现人工智能。自然语言处理是指让计算机能够理解和处理自然语言，实现人机交互和智能对话的能力。除此之外，还需要结合相关的算法和技术，如深度学习、模式识别、知识图谱等，来实现计算机的智能化。'}

In [47]:
m_overall_chain.invoke({"subject":"汉语文学"})



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
你是一位学生。根据传入的学科，提出一个对应学科的知识问题。

学科：汉语文学
问题：以下是关于上述学科的一个只是问题：

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
你是一位学科老师。根据传入的学科问题，你需要做出该学科问题的答案。

学科问题：
古代中国诗歌的发展历史包括哪些时期？

来自学科老师对上述问题的答案：

> Finished chain.

> Finished chain.


{'subject': '汉语文学',
 'synopsis': '古代中国诗歌的发展历史包括哪些时期？',
 'review': '\n古代中国诗歌的发展历史主要可以分为两个时期：先秦时期和汉唐时期。先秦时期，主要是指战国时期以前，诗歌形式以诗经为主，内容多为国家政治和社会风俗。而汉唐时期，是指汉代和唐代，这一时期的诗歌形式和内容都有了较大的发展，出现了大量优秀的诗人和诗作，也是中国诗歌发展的黄金时期。'}